# Hardpoint Breakaway Tests - Recent History

This notebook evaluates the hardpoint breakaway tests performed on a given `day_obs`.  

In [ ]:
# Times Square Parameters
day_obs = 20250804  # YYYYMMDD

In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

import asyncio
import ipywidgets as W
import matplotlib.dates as mdates
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from astropy.time import Time
from typing import Optional
from io import BytesIO
from IPython.display import display, clear_output
from typing import List

from lsst_efd_client import EfdClient
from lsst.ts.xml.enums.MTM1M3 import HardpointTest
from lsst.summit.utils.efdUtils import (
    getEfdData,
    getDayObsEndTime,
    getDayObsStartTime,
    makeEfdClient,
)

In [ ]:
# HP BreakAway Limits from https://sitcomtn-082.lsst.io/
COMPRESSION_LOWER_LIMIT = 2981  # N
COMPRESSION_UPPER_LIMIT = 3959  # N
TENSION_LOWER_LIMIT = -4420  # N
TENSION_UPPER_LIMIT = -3456  # N

# Used to select the window size near zero measured forces.
DISPLACEMENT_CROP_RANGE = 300  # um
DISPLACEMENT_CROP_RANGE_FOR_FIT = 100  # um

# Used to determine if a test was valid or not
MEASURED_FORCE_MAXIMUM_TOLERANCE = 1000  # N

# Meter to micrometer
METER_TO_MICROMETER = 1e6  # um/m

# Maximum spec stiffness
SPEC_STIFFNESS = 100  # N/um

# Status Plotting
STATUS_COLORS = {
    HardpointTest.NOTTESTED: "lightgrey",
    HardpointTest.MOVINGNEGATIVE: "lightsteelblue",
    HardpointTest.TESTINGPOSITIVE: "forestgreen",
    HardpointTest.TESTINGNEGATIVE: "royalblue",
    HardpointTest.MOVINGREFERENCE: "darkseagreen",
    HardpointTest.PASSED: "dimgrey",
    HardpointTest.FAILED: "red",
}

# We use this a lot, so let's make it shorter
TEST_POS_COLOR = STATUS_COLORS[HardpointTest.TESTINGPOSITIVE]
TEST_NEG_COLOR = STATUS_COLORS[HardpointTest.TESTINGNEGATIVE]

# Number of hardpoints
n_hardpoints = 6

# Create an EFD client instance
client = makeEfdClient()

# Get the start and end times
start_time = getDayObsStartTime(day_obs)
end_time = getDayObsEndTime(day_obs)

In [ ]:
# Query the CSC level command that we can use to define the start of a
# breakaway test for a given strut.
test_command_df = getEfdData(
    client=client,
    topic="lsst.sal.MTM1M3.command_testHardpoint",
    columns=["hardpointActuator"],
    begin=start_time,
    end=end_time,
)

In [ ]:
# Query the status of the breakaway tests as they evolve.
test_status_df = getEfdData(
    client=client,
    topic="lsst.sal.MTM1M3.logevent_hardpointTestStatus",
    columns=[f"testState{i}" for i in range(n_hardpoints)],
    begin=start_time,
    end=end_time,
)

In [ ]:
def add_custom_time_format(ax: plt.Axes) -> None:
    """Convenient function to deal with time xaxis"""
    locator = mdates.MinuteLocator(interval=1)
    ax.xaxis.set_major_locator(locator)

    formatter = mdates.DateFormatter("%H:%M")
    ax.xaxis.set_major_formatter(formatter)


def add_plot_cosmetics(
    ax: plt.Axes, x_label: str, y_label: str, title: str | None = None, right_axis: bool = False
) -> None:
    """Convenient function to deal with cosmetics applied to every plot"""
    ax.set_xlabel(x_label)
    ax.set_ylabel(y_label)
    ax.set_ylim(TENSION_LOWER_LIMIT - 500, COMPRESSION_UPPER_LIMIT + 500)
    ax.grid(":", alpha=0.2)

    if title:
        ax.set_title(title)

    if right_axis:
        ax.yaxis.set_label_position("right")
        ax.yaxis.tick_right()


def add_ranges_where_breakaway_should_happen(ax: plt.Axes) -> None:
    """
    Add two shaded areas representing where the breakaway mechanism
    must be triggered.
    """
    ax.axhspan(COMPRESSION_LOWER_LIMIT, COMPRESSION_UPPER_LIMIT, fc=TEST_POS_COLOR, alpha=0.2)
    ax.axhspan(TENSION_LOWER_LIMIT, TENSION_UPPER_LIMIT, fc=TEST_NEG_COLOR, alpha=0.2)


def build_status_timeline(
    start_time: pd.Timestamp, status: pd.DataFrame, end_reasons: List[int] | None = None
) -> pd.DataFrame:
    """
    Create a new dataframe containing the time windows associated with status
    events.
    """
    if end_reasons is None:
        end_reasons = [
            HardpointTest.NOTTESTED,
            HardpointTest.PASSED,
            HardpointTest.FAILED,
        ]

    slice = status.loc[start_time:]
    end_time = find_end_of_test(slice, end_reasons)
    end_time = (
        pd.Timestamp(end_time) + pd.Timedelta("1s") if end_time is not None else None
    )
    slice = slice.loc[:end_time] if end_time is not None else slice
    print(f"Start time: {start_time}\nEnd Time: {end_time}")

    timeline = pd.DataFrame(
        index=[hp.name for hp in HardpointTest],
        columns=[col for col in status.columns if col != "index"],
    )

    for col in status.columns:
        for hp in HardpointTest:
            mask = slice[col].isin([hp])
            tstamp = slice[col][mask].first_valid_index()
            timeline.loc[hp.name, col] = tstamp if tstamp is not None else pd.NaT

    return timeline


def create_status_table(df: pd.DataFrame, idx: int) -> pd.DataFrame:
    """
    Create a new dataframe containing the time windows associated with status

    Paramenters
    -----------
    df : pd.DataFrame
        Table containing the status published by events.
    idx : int
        0-based strut number.
    """
    column_name = df[f"testState{idx}"]
    segments = get_status_segments(column_name, df.index.min(), df.index.max())

    df = pd.DataFrame(segments, columns=["start", "end", "status"])
    df["status"] = df["status"].apply(lambda x: HardpointTest(x))

    return df


def build_labels(cmds: pd.DataFrame, sts: pd.DataFrame) -> pd.DataFrame:
    """
    Once our dataframe has a new column called "group_id", we can use it to
    group our tests by this identifier.
    """
    summary = (
        cmds.reset_index()
        .groupby("group_id")
        .agg(
            reference_time=("index", lambda s: s.dt.floor("min").min()),
            n=("group_id", "size"),
            uniq_hp=("hardpointActuator", lambda s: sorted(s.unique())),
        )
        .reset_index()
    )

    summary["start_time"] = summary["reference_time"].apply(
        lambda x: sts.index[sts.index.searchsorted(x, side="left")]
    )

    summary["end_time"] = summary["reference_time"].apply(
        lambda x: find_end_of_test(sts, x)
    )

    return summary


def ensure_utc(df: pd.DataFrame) -> pd.DataFrame:
    """
    Problems with timezones are common when querying data from the EFD.
    This function ensures that we localize the timestamps.
    """
    if df.index.tz is None:
        return df.tz_localize("UTC")
    else:
        return df.tz_convert("UTC")


def find_end_of_test(
    df: pd.DataFrame,
    start_time: pd.Timestamp,
    end_reasons: List[int] | None = None,
    end_buffer: str = "10s",
) -> Optional[pd.Timestamp]:
    """
    Find the end of a test run by looking for the first row where all columns
    have the same value and that value is in end_reasons.
    """
    if end_reasons is None:
        end_reasons = [
            HardpointTest.NOTTESTED,
            HardpointTest.PASSED,
            HardpointTest.FAILED,
        ]

    # Ensure we don't get a fake NOTTESTED in the beginning of the test
    df = df.loc[df.index >= start_time + pd.Timedelta("5s")]

    for idx, row in df.iterrows():
        unique_values = set(row.values)
        if len(unique_values) == 1 and list(unique_values)[0] in end_reasons:
            return idx + pd.Timedelta(end_buffer)

    return None


def get_status_segments(status_s: pd.Series, t0: pd.Timestamp, t1: pd.Timestamp):
    """Return list of (start, end, state) where state is constant in [t0,t1]."""
    s = status_s.sort_index()
    s = s[(s.index <= t1) & (s.index >= (t0 - pd.Timedelta("1s")))]

    if s.empty:
        return []
    if s.index[0] > t0:
        s = pd.concat([pd.Series([s.iloc[0]], index=[t0]), s])
    if s.index[-1] < t1:
        s.loc[t1] = s.iloc[-1]

    s = s.sort_index()
    changed = s.ne(s.shift()).to_numpy()
    segs = []
    idxs = [i for i, c in enumerate(changed) if c]
    idxs.append(len(s) - 1)

    for i in range(len(idxs) - 1):
        a = s.index[idxs[i]]
        b = s.index[idxs[i + 1]]
        state = int(s.iloc[idxs[i]])
        beg = max(pd.Timestamp(a), t0)
        end = min(pd.Timestamp(b), t1)
        if end > beg:
            segs.append((beg, end, state))

    return segs


def group_by_gaps(gap: str, df: pd.DataFrame) -> pd.DataFrame:
    """
    It is uncommon to have more than one execution of hardpoint breakaway
    tests in a single day. But hey happen. This function defines group labels
    that we can use to distinguish between each run of this test.
    """
    time_gap = pd.Timedelta(gap)
    group_id = df.index.to_series().diff().gt(time_gap).cumsum()
    return df.assign(group_id=group_id)


def query_hardpoints_telemetry(
    client: EfdClient, summary_row: pd.Series, sampling: str = "100ms"
) -> pd.DataFrame:
    start_time = Time(summary_row["start_time"]).isot
    end_time = Time(summary_row["end_time"]).isot

    columns_displacement = [
        f"mean(displacement{i - 1}) as mean_displacement_{i - 1}"
        for i in summary_row["uniq_hp"]
    ]
    columns_forces = [
        f"mean(measuredForce{i - 1}) as mean_measured_force_{i - 1}"
        for i in summary_row["uniq_hp"]
    ]
    columns = ", ".join(columns_displacement + columns_forces)

    query = f"""
        SELECT {columns}
        FROM "lsst.sal.MTM1M3.hardpointActuatorData"
        WHERE time >= '{start_time}Z'
        AND time <= '{end_time}Z'
        GROUP by time({sampling})
    """

    # Perform the query
    telemetry = asyncio.run(client.influx_client.query(query))

    # Convert displacement from m to um
    for c in columns_displacement:
        c = c.split(" ")[-1]
        telemetry[c] = telemetry[c].astype(float) * METER_TO_MICROMETER

    return telemetry


def plot_forces_timeline(
    ax: plt.Axes,
    tel: pd.DataFrame,
    hpi: int, 
    test_positive: pd.DataFrame | None = None,
    test_negative: pd.DataFrame | None = None
) -> List[pd.DataFrame]:
    """
    Plot the telemetry for a single hardpoint actuator.
    """
    ax.clear()
    col_force = f"mean_measured_force_{hpi}"
    if col_force in tel.columns:
        ax.plot(tel.index, tel[col_force], color="black")

        if test_positive is not None:
            ax.plot(test_positive.index, test_positive[col_force], color=TEST_POS_COLOR, label="Testing Positive")

        if test_negative is not None:
            ax.plot(test_negative.index, test_negative[col_force], color=TEST_NEG_COLOR, label="Testing Negative")

    add_plot_cosmetics(
        ax, "Time [UTC]", "Measured Force [N]", "Measured Force Timeline"
    )
    add_ranges_where_breakaway_should_happen(ax)
    add_custom_time_format(ax)
    ax.legend(fontsize=9)


def plot_status_timeline(
    ax, sub_status_df: pd.DataFrame, t0: pd.Timestamp, t1: pd.Timestamp
):
    """Plot colored boxes per strut vs time for states 1..7 in [t0,t1]."""
    ax.clear()
    ax.set_xlim(t0, t1)
    ax.set_ylim(0.5, 6.5)
    ax.set_yticks([1, 2, 3, 4, 5, 6])
    ax.set_yticklabels([f"HP{i}" for i in range(1, 7)])
    ax.set_xlabel("Time (UTC)")
    ax.set_ylabel("Strut")
    ax.xaxis.set_major_formatter(mdates.DateFormatter("%H:%M"))

    for i in range(6):
        col = f"testState{i}"
        if col not in sub_status_df.columns:
            continue
        segs = get_status_segments(sub_status_df[col], t0, t1)
        y = i + 1
        for a, b, state in segs:
            enum_state = HardpointTest(state)
            ax.broken_barh(
                [(mdates.date2num(a), mdates.date2num(b) - mdates.date2num(a))],
                (y - 0.4, 0.8),
                facecolors=STATUS_COLORS.get(enum_state, "#bdbdbd"),
                edgecolors="none",
            )

    handles = [
        plt.Line2D(
            [0], [0], lw=8, color=STATUS_COLORS[st], label=f"{st.value}-{st.name}"
        )
        for st in HardpointTest
    ]
    ax.legend(
        handles=handles, bbox_to_anchor=(1.02, 1), loc="upper left", borderaxespad=0.0
    )
    ax.grid(True, linestyle=":", alpha=0.3)


def plot_stiffness(ax: plt.Axes, test_positive: pd.DataFrame, test_negative: pd.DataFrame, hpi: int):
    """
    Plot the measured forces versus the displacement for when moving the
    hardpoint in the positive (compression) direction and in the negative
    (tension). The slant of the curve near zero corresponds to the stiffness of
    the hardpoint in that direction.
    """
    # First, let's add the requirement
    add_ranges_where_breakaway_should_happen(ax)
    measured_stiffness = []

    for df, direction in zip([test_positive, test_negative], ["Positive", "Negative"]):
        force_col = f"mean_measured_force_{hpi}"
        disp_col = f"mean_displacement_{hpi}"

        # Find where the forces are minimum
        min_forces_index = df[force_col].abs().idxmin()
        displacement_where_force_is_minimum = df.loc[min_forces_index, disp_col]

        if df[force_col].abs().min() > MEASURED_FORCE_MAXIMUM_TOLERANCE:
            print(f"Warning - Could not find valid test data for HP{hpi+1}")
            measured_stiffness.append(np.nan)
            continue

        # Put the zero of the displacement near the minimum measured force
        df.loc[:, disp_col] = df.loc[:, disp_col] - displacement_where_force_is_minimum

        # Filter out based on the displacement values
        df = df[df[disp_col].abs() <= DISPLACEMENT_CROP_RANGE]
        
        # Plot the telemetry
        color = TEST_POS_COLOR if direction == "Positive" else TEST_NEG_COLOR
        ax.plot(df[disp_col], df[force_col], "-", color=color, label=f"Test {direction}", alpha=0.5)
        
        # Fit the stiffness using a polynomium
        df_for_fit = df[df[disp_col].abs() <= DISPLACEMENT_CROP_RANGE_FOR_FIT]
        poly = np.polyfit(df_for_fit[disp_col], df_for_fit[force_col], deg=1)
        ax.plot(df[disp_col], np.polyval(poly, df[disp_col]),
                "--", color=color, label=f"Fit {direction} - Stiff. [{np.round(poly[0]):.0f} N/um]")
        
        measured_stiffness.append(np.round(poly[0]))

    # Plot requirement
    ax.plot(df[disp_col], df[disp_col] * SPEC_STIFFNESS, ":", color="k", label=f"Spec Stiff. [{SPEC_STIFFNESS} N/um]")
    ax.legend(fontsize=9)
    add_plot_cosmetics(ax, "Displacement [um]", "Measured Force [N]", right_axis=True)
    return measured_stiffness


def get_timestamp_from_dataframe(df: pd.DataFrame, column_name: str) -> pd.Timestamp:
    """
    Retrieve timestamps from a dataframe.
    The `column_name` is expected to be a column that contains timestamps.
    """
    return round_seconds(df[column_name].iloc[0])


def round_seconds(obj: pd.Timestamp) -> pd.Timestamp:
    """
    Round the timestamps to seconds.
    """
    if obj.microsecond >= 500000:
        obj += pd.Timedelta(seconds=1)
    return obj.replace(microsecond=0)


def select_region_where_forces_are_changing(
    df: pd.DataFrame, column_start: str, column_end: str
) -> pd.DataFrame:
    """Filter out the regions where the forces are changing in a linear fashion"""
    # Get the timestamps near the interesting region
    time_start = get_timestamp_from_dataframe(df, column_start)
    time_end = get_timestamp_from_dataframe(df, column_end)

    # Filter out our dataframe considering the time
    df = df[(df.index >= time_start) * (df.index <= time_end)].copy()

    # Find force intersection nearest to zero
    minimum_measured_force_index = df.measured_force.abs().idxmin()
    displacement_where_force_is_minimum = df.displacement.loc[
        minimum_measured_force_index
    ]

    if df.measured_force.abs().min() > MEASURED_FORCE_MAXIMUM_TOLERANCE:
        reason = df.end_reason.iloc[0]
        hp = df.hardpoint.iloc[0]
        print(
            f"Warning - Could not find valid test data "
            f"when {HardpointTest(reason).name} for HP{hp} "
            f"found minimum absolute measured forces {df.measured_force.abs().min()}."
        )

    # Put the zero of the displacement near the minimum measured force
    df.displacement = df.displacement - displacement_where_force_is_minimum

    # Filter out based on the displacement values
    df = df[df.displacement.abs() <= DISPLACEMENT_CROP_RANGE]

    return df


def slice_telemetry(tel: pd.DataFrame, status: pd.DataFrame, state: int) -> pd.DataFrame:
    """
    Slice the telemetry dataframe to only include the time range where the
    hardpoint is in the expected state
    """
    temp_df = status[status["status"] == state].iloc[0]
    return tel.loc[temp_df.start:temp_df.end, :]



In [ ]:
def make_hardpoint_group_selector(
    commands: pd.DataFrame,
    statuses: pd.DataFrame,
    gap: str = "3min",
):
    """Select a group, simulate query, and plot status timeline boxes."""
    statuses = ensure_utc(statuses)
    commands = group_by_gaps(gap, ensure_utc(commands))
    summary = build_labels(commands, statuses)

    def _label(row: pd.Series) -> str:
        s = row["reference_time"].strftime("%Y-%m-%d %H:%M UTC")
        return f"Test run {int(row['group_id'])} | Started near {s} | {row['n']} cmds"

    options = [(_label(r), int(r["group_id"])) for _, r in summary.iterrows()]
    dd = W.Dropdown(description="Select test run:", options=options,
                    value=options[0][1] if options else None,
                    layout=W.Layout(width="650px"))

    out = W.Output()

    def show_group(gid: int):
        with out:
            
            # Clear the output and print a nice message for impacient people
            clear_output()
            print("Querying and processing data...")
            row = summary.loc[summary["group_id"] == gid].iloc[0]
            t0, t1 = row["start_time"], row["end_time"]
            
            sub_status = test_status_df.loc[t0:t1]
            fig, ax = plt.subplots(1, 1, figsize=(12, 4), constrained_layout=True)
            plot_status_timeline(ax, sub_status, t0, t1)
                        
            # display the figure using html
            display(W.HTML(f"<b>Group window:</b> {t0} → {t1} (UTC)"))
            display(fig)
            plt.close(fig)  # Ensures the plots are shown only once

            telemetry = query_hardpoints_telemetry(client, row)

            for hp in row["uniq_hp"]:
                
                status_df = create_status_table(sub_status, hp-1)
                pos_tel = slice_telemetry(telemetry, status_df, HardpointTest.TESTINGPOSITIVE)
                neg_tel = slice_telemetry(telemetry, status_df, HardpointTest.TESTINGNEGATIVE)

                fig, (ax_t, ax_s) = plt.subplots(1, 2, figsize=(12, 4), constrained_layout=True)
                fig.suptitle(f"Hardpoint {hp} telemetry")
                
                plot_forces_timeline(ax_t, telemetry, hp-1, test_positive=pos_tel, test_negative=neg_tel)
                measured_stiffness = plot_stiffness(ax_s, pos_tel, neg_tel, hp-1)
                
                display(W.HTML(f"""
                    <b>Hardpoint {hp}: </b><br>
                    Positive (compression) breakaway happened at
                    <span style="color: {TEST_POS_COLOR}"><b>
                      {pos_tel[f"mean_measured_force_{hp-1}"].max():.0f} N
                    </b></span><br>
                    Negative (tension) breakaway happened at
                    <span style="color: {TEST_NEG_COLOR}"><b>
                      {neg_tel[f"mean_measured_force_{hp-1}"].min():.0f} N
                    </b></span><br>
                    Stiffness measured in positive (compression): <b>{measured_stiffness[0]:.0f} N/um</b> <br>
                    Stiffness measured in negative (tension): <b>{measured_stiffness[1]:.0f} N/um</b>
                """))
                
                display(fig)
                plt.close(fig)

    def _on_change(change):
        if change["name"] == "value" and change["new"] is not None:
            show_group(change["new"])

    dd.observe(_on_change)
    display(dd, out)
    if dd.value is not None:
        show_group(dd.value)

    return dd, out


# Example usage ---
dd, out = make_hardpoint_group_selector(test_command_df, test_status_df)
